In [1]:
# !pip install openai groq langdetect icalendar google-api-python-client google-auth-oauthlib slack_sdk discord.py python-dateutil pytz nest_asyncio

In [2]:
# pip install dateparser

In [3]:
import requests
url = "http://localhost:3000/v1/chat/completions"
headers = {"Content-Type": "application/json"}
data = {
    "model": "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat",
    "messages": [
        {"role": "system", "content": "You are an expert in scheduling."},
        {"role": "user", "content": "Parse: 'Hi team, let’s meet next Thursday'"}
    ],
    "stream": False,
    "max_tokens": 128
}
response = requests.post(url, headers=headers, json=data)
print(response.json())

{'id': 'chatcmpl-a15a726a987844eb9ea69ece1e08980f', 'object': 'chat.completion', 'created': 1752397515, 'model': '/home/user/Models/deepseek-ai/deepseek-llm-7b-chat', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'reasoning_content': None, 'content': 'The sentence "Hi team, let\'s meet next Thursday" can be broken down into its individual components and parsed for meaning and information.\n\n1. "Hi team" is a greeting and introduction to the topic of the message. It is used to acknowledge the presence of the team members and signal the beginning of a discussion.\n\n2. "let’s meet" is the main action or suggestion being proposed. It is a request for the team members to gather together for a discussion or meeting.\n\n3. "next Thursday" is the specific time and date being proposed for the meeting. It is used to provide', 'tool_calls': []}, 'logprobs': None, 'finish_reason': 'length', 'stop_reason': None}], 'usage': {'prompt_tokens': 29, 'total_tokens': 157, 'completion_tokens'

In [ ]:
import json
import os
from datetime import datetime, timedelta
from dateutil import parser
import pytz
import uuid
import logging
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from openai import OpenAI
from flask import Flask, request, jsonify
from threading import Thread
from pydantic import BaseModel, EmailStr, Field
import discord
from discord.ext import commands
from slack_sdk import WebClient
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from functools import lru_cache
import re
import asyncio
import nest_asyncio
from icalendar import Calendar, Event as IcsEvent
from langdetect import detect
from groq import Groq
import time
import dateparser
from datetime import datetime, timedelta
import datetime, re, logging
import dateutil.parser


# Enable asyncio in Jupyter Notebook
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# vLLM configuration
BASE_URL = "http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"
client = OpenAI(api_key="NULL", base_url="http://localhost:3000/v1")



# Groq fallback configuration
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "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")
groq_client = Groq(api_key=GROQ_API_KEY)

# Flask setup
app = Flask(__name__)
request_queue = Queue()
received_data = []

# Notification setup
DISCORD_TOKEN = os.getenv("DISCORD_TOKEN")
SLACK_TOKEN = os.getenv("SLACK_TOKEN")
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_PASS = os.getenv("EMAIL_PASS")
DISCORD_CHANNEL = os.getenv("DISCORD_CHANNEL", None)
EMAIL_SERVER = None
slack_client = None
discord_bot = None

if EMAIL_USER and EMAIL_PASS:
    try:
        EMAIL_SERVER = smtplib.SMTP('smtp.gmail.com', 587)
        EMAIL_SERVER.starttls()
    except Exception as e:
        logging.warning(f"Failed to initialize email server: {e}")

if SLACK_TOKEN:
    try:
        slack_client = WebClient(token=SLACK_TOKEN)
    except Exception as e:
        logging.warning(f"Failed to initialize Slack client: {e}")

if DISCORD_TOKEN and DISCORD_CHANNEL:
    try:
        discord_bot = commands.Bot(command_prefix='!')
        DISCORD_CHANNEL = int(DISCORD_CHANNEL)
    except Exception as e:
        logging.warning(f"Failed to initialize Discord bot: {e}")

# Location to time zone mapping
LOCATION_TO_TIMEZONE = {
    'IIT Mumbai': 'Asia/Kolkata',
    'New York': 'America/New_York',
    'London': 'Europe/London',
    'Online': 'Asia/Kolkata'  # Default for virtual meetings
}

# Custom tool for current date/time
def get_current_date() -> str:
    """Return the current date/time as an ISO-formatted string."""
    return datetime.now().isoformat()

In [5]:
class AttendeeInput(BaseModel):
    email: EmailStr

class MeetingRequest(BaseModel):
    Request_id: str = Field(default_factory=lambda: str(uuid.uuid4()), pattern=r"^[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}$")
    Datetime: str = Field(default_factory=lambda: datetime.now().strftime("%d-%m-%YT%H:%M:%S"), pattern=r"^\d{2}-\d{2}-\d{4}T\d{2}:\d{2}:\d{2}$")
    Location: str = "Online"
    From: EmailStr
    Attendees: list[AttendeeInput] = []
    Subject: str = "Meeting"
    EmailContent: str = ""

class Event(BaseModel):
    StartTime: str
    EndTime: str
    NumAttendees: int
    Attendees: list[str]
    Summary: str

class AttendeeOutput(BaseModel):
    email: EmailStr
    events: list[Event] = []

    
class MeetingResponse(BaseModel):
    Request_id: str
    Datetime: str
    Location: str
    From: EmailStr
    Attendees: list[AttendeeOutput]
    Subject: str
    EmailContent: str
    EventStart: str
    EventEnd: str
    Duration_mins: str
    MetaData: dict

class AssistantResponse(BaseModel):
    processed: bool
    output: MeetingResponse

In [6]:
import dateparser
import pytz
from datetime import datetime, timedelta

def make_base_datetime(dt_str):
    """
    Parse either DD-MM-YYYYThh:mm:ss or ISO8601 into a timezone-aware Asia/Kolkata datetime.
    """
    try:
        # first, try your custom DD-MM-YYYY
        base = datetime.strptime(dt_str, "%d-%m-%YT%H:%M:%S")
    except ValueError:
        # fallback to generic ISO parse (e.g. 2025-07-12T22:50:00 or with offset)
        base = datetime.fromisoformat(dt_str)
    # ensure tz‑aware in IST
    if base.tzinfo is None:
        base = pytz.timezone("Asia/Kolkata").localize(base)
    return base


In [7]:
import re
import logging
import dateparser
from datetime import timedelta, datetime
from dateutil.parser import isoparse
import pytz
import json

class AI_AGENT:
    def __init__(self, vllm_client, groq_client, model_path):
        self.vllm_client = vllm_client
        self.groq_client = groq_client
        self.model_path = model_path

    @staticmethod
    def extract_json_from_text(text):
        """Extract the first JSON object from a block of text."""
        match = re.search(r"\{.*?\}", text, re.DOTALL)
        if match:
            return json.loads(match.group())
        raise json.JSONDecodeError("No JSON found in response", text, 0)

    @staticmethod
    def parse_email(email_content, default_dt_str):
        """Parse email content to extract meeting details with improved date parsing."""
        try:
            # 1) Base datetime and dateparser settings
            base = make_base_datetime(default_dt_str)
            settings = {
                "RELATIVE_BASE": base,
                "PREFER_DATES_FROM": "future",
                "TIMEZONE": "Asia/Kolkata",
                "RETURN_AS_TIMEZONE_AWARE": True,
                "PREFER_DAY_OF_MONTH": "first"
            }
        
            # 2) Regex patterns
            time_patterns = [
                r'(?:next\s+)?(?:monday|tuesday|wednesday|thursday|friday|saturday|sunday)\s+at\s+\d{1,2}(?::\d{2})?\s*(?:am|pm|a\.m\.|p\.m\.)?',
                r'(?:next\s+)?(?:monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b',
                r'(?:tomorrow|tmrw)\s+at\s+\d{1,2}(?::\d{2})?\s*(?:am|pm|a\.m\.|p\.m\.)?',
                r'(?:tomorrow|tmrw)\b',
                r'\d{1,2}(?::\d{2})?\s*(?:am|pm|a\.m\.|p\.m\.)',
                r'(?:morning|afternoon|evening)',
                r'asap|urgent|immediately'
            ]
        
            extracted = None
            for pat in time_patterns:
                m = re.search(pat, email_content.lower())
                if m:
                    extracted = m.group(0)
                    break
        
            # 3) Initial parse
            dt = dateparser.parse(extracted, settings=settings) if extracted else \
                 dateparser.parse(email_content, settings=settings)
        
            # 4) Fallback for missing or past times
            if not dt or dt.date() < base.date():
                dt = base + timedelta(days=1)
                dt = dt.replace(hour=10, minute=30, second=0, microsecond=0)
        
            # 5) Fix "next <weekday>" logic
            if extracted and re.match(r'next\s+(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b', extracted, flags=re.IGNORECASE):
                weekday_map = {
                    'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
                    'friday': 4, 'saturday': 5, 'sunday': 6
                }
                target_weekday = weekday_map[re.search(r'next\s+(\w+)', extracted, re.IGNORECASE).group(1).lower()]
                days_ahead = target_weekday - base.weekday()
                # Always schedule for the next week for "next <weekday>"
                days_ahead += 7
                dt = base + timedelta(days=days_ahead)
                dt = dt.replace(hour=10, minute=30, second=0, microsecond=0)  # Default to 9 AM
        
            # 6) Handle specific time if provided
            if extracted and re.search(r'\d{1,2}(?::\d{2})?\s*(?:am|pm|a\.m\.|p\.m\.)', extracted, flags=re.IGNORECASE):
                parsed_time = dateparser.parse(extracted, settings=settings)
                if parsed_time:
                    dt = dt.replace(hour=parsed_time.hour, minute=parsed_time.minute)
        
            # 7) "tomorrow" bare → set to 9 AM
            if extracted and re.fullmatch(r'(?:tomorrow|tmrw)\b', extracted, flags=re.IGNORECASE):
                dt = dt.replace(hour=10, minute=30, second=0, microsecond=0)        
            # 8) Ensure working hours
            if not is_working_hours(dt, pytz.timezone("Asia/Kolkata")):
                logging.warning(f"Parsed time {dt} is outside working hours; defaulting to 9 AM next day")
                dt = dt.replace(hour=9, minute=0, second=0, microsecond=0) + timedelta(days=1)
                # Skip to next weekday if the new day is a weekend
                while dt.weekday() >= 5:  # Saturday (5) or Sunday (6)
                    dt += timedelta(days=1)
        
            # 9) Duration (default 30 min)
            dm = re.search(r'(\d+)\s*(?:min|minutes|hour|hours|hr|hrs)', email_content.lower())
            if dm:
                n = int(dm.group(1))
                duration = n * 60 if 'hour' in dm.group(0) or 'hr' in dm.group(0) else n
            else:
                duration = 30
        
            # 10) Preferences
            prefs = []
            lc = email_content.lower()
            if 'morning' in lc: prefs.append('prefer mornings')
            if 'afternoon' in lc: prefs.append('prefer afternoons')
            if 'evening' in lc: prefs.append('prefer evenings')
        
            # 11) Build start/end
            start = dt
            end = start + timedelta(minutes=duration)
        
            # 12) Force 9 AM on bare date + "prefer mornings"
            if 'prefer mornings' in prefs and extracted and not re.search(r'\d{1,2}', extracted):
                start = start.replace(hour=9, minute=0, second=0, microsecond=0)
                end = start + timedelta(minutes=duration)
        
            return {
                "start": start,
                "end": end,
                "duration_mins": duration,
                "preferences": ", ".join(prefs)
            }
        
        except Exception as e:
            logging.warning(f"Error parsing email time: {e}")
            fallback = make_base_datetime(default_dt_str) + timedelta(hours=1)
            return {
                "start": fallback,
                "end": fallback + timedelta(minutes=30),
                "duration_mins": 30,
                "preferences": ""
            }

    def generate_reschedule_message(self, conflict_details, lang: str = "en"):
        """Generate a polite rescheduling message in the specified language."""
        try:
            prompt = f"""
            You are an expert in scheduling. Generate a polite, professional rescheduling message in {lang} for a meeting due to conflicts: {conflict_details}.
            - Keep the tone friendly and concise.
            - Suggest proposing alternative times if no common slot is found, or confirm the new time if a slot is selected.
            - Avoid regex, special characters, or technical jargon in the message.
            - Return only the message text, no JSON or extra formatting.
            """
            try:
                response = self.vllm_client.chat.completions.create(
                    model=self.model_path,
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=128,
                    temperature=0.2
                )
                content = (
                    response.choices[0].message.get("content")
                    if response and response.choices and isinstance(response.choices[0].message, dict)
                    else getattr(response.choices[0].message, "content", None)
                    if response and response.choices
                    else None
                )
                return content.strip() if content else "Due to scheduling conflicts, please propose alternative times."
            except Exception as e:
                logging.warning(f"vLLM failed, using Groq fallback: {e}")
                response = self.groq_client.chat.completions.create(
                    model="llama3-8b-8192",
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=128,
                    temperature=0.2
                )
                content = (
                    response.choices[0].message.get("content")
                    if response and response.choices and isinstance(response.choices[0].message, dict)
                    else getattr(response.choices[0].message, "content", None)
                    if response and response.choices
                    else None
                )
                return content.strip() if content else "Due to scheduling conflicts, please propose alternative times."
        except Exception as e:
            logging.error(f"Error generating reschedule message: {e}")
            return "Due to scheduling conflicts, please propose alternative times." if lang == "en" else "Due to scheduling conflicts, please suggest other times." if lang == "hi" else "Please propose alternative times due to scheduling conflicts."

In [8]:
from datetime import datetime, timedelta
import asyncio
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache
import logging
import time

@lru_cache(maxsize=100)
def retrieve_calendar_events(email: str, start_time: str, end_time: str):
    """Fetch calendar events with retry logic."""
    events_list = []
    start = datetime.fromisoformat(start_time)
    end = datetime.fromisoformat(end_time)
    
    for attempt in range(3):
        try:
            creds = Credentials.from_authorized_user_file(f"../Keys/{email.split('@')[0]}.token")
            service = build("calendar", "v3", credentials=creds)
            events_result = service.events().list(
                calendarId='primary',
                timeMin=start.isoformat(),
                timeMax=end.isoformat(),
                singleEvents=True,
                orderBy='startTime'
            ).execute()
            events = events_result.get('items', [])
            time_zone = 'Asia/Kolkata'
            
            for event in events:
                attendee_list = [attendee['email'] for attendee in event.get('attendees', [{'email': 'SELF'}])]
                
                # Handle both timed and all-day events
                start_event = event.get('start', {})
                end_event = event.get('end', {})
                
                # Get timezone from event or use default
                time_zone = start_event.get('timeZone', time_zone)
                
                # Handle dateTime vs date fields
                if 'dateTime' in start_event:
                    start_time_str = start_event['dateTime']
                    end_time_str = end_event['dateTime']
                elif 'date' in start_event:
                    # All-day events - convert to datetime with timezone
                    start_date = start_event['date']
                    end_date = end_event['date']
                    # Convert date to datetime string with timezone
                    start_time_str = f"{start_date}T00:00:00+05:30"
                    end_time_str = f"{end_date}T00:00:00+05:30"
                else:
                    # Skip events without proper time information
                    logging.warning(f"Event {event.get('summary', 'Untitled')} has no valid time information")
                    continue
                
                events_list.append({
                    "StartTime": start_time_str,
                    "EndTime": end_time_str,
                    "NumAttendees": len(set(attendee_list)),
                    "Attendees": list(set(attendee_list)),
                    "Summary": event.get('summary', 'Untitled Event')
                })
            return events_list, time_zone
        except Exception as e:
            logging.warning(f"Retry {attempt+1}/3 for {email}: {e}")
            time.sleep(2 ** attempt)  # Exponential backoff
    logging.error(f"Failed to fetch calendar events for {email}")
    return [], 'Asia/Kolkata'

def calculate_meeting_date_range(request_datetime: str, target_day: str = None,tz_str: str = "Asia/Kolkata"):
    """Calculate focused date range for calendar queries based on meeting request."""
    # Parse the request datetime
    # Handle format: "12-07-2025T12:34:55"
    try:
        if 'T' in request_datetime:
            date_part, time_part = request_datetime.split('T')
            day, month, year = date_part.split('-')
            request_date = datetime(int(year), int(month), int(day))
        else:
            # Handle other formats if needed
            request_date = datetime.fromisoformat(request_datetime)
    except Exception as e:
        logging.warning(f"Error parsing datetime {request_datetime}: {e}")
        # Default to today
        request_date = datetime.now()

    # If target_day is specified (like "Thursday"), find the next occurrence
    if target_day:
        day_mapping = {
            'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
            'friday': 4, 'saturday': 5, 'sunday': 6
        }
        target_weekday = day_mapping.get(target_day.lower())
        if target_weekday is not None:
            current_weekday = request_date.weekday()
            days_ahead = target_weekday - current_weekday
            if days_ahead <= 0:  # Target day already passed this week
                days_ahead += 7
            target_date = request_date + timedelta(days=days_ahead)
        else:
            target_date = request_date
    else:
        target_date = request_date

    # Return just the target day range (start of day to end of day)
    import pytz
    tz = pytz.timezone(tz_str)
    start_naive = target_date.replace(hour=0, minute=0, second=0, microsecond=0)
    end_naive   = target_date.replace(hour=23, minute=59, second=59, microsecond=999999)
    start_of_day = tz.localize(start_naive)
    end_of_day   = tz.localize(end_naive)
    return start_of_day, end_of_day
    

def extract_target_day_from_email(email_content: str):
    """Extract target day from email content."""
    email_lower = email_content.lower()
    days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    
    for day in days:
        if day in email_lower:
            return day
    return None

async def fetch_all_events(attendees, start_time, end_time):
    """Async fetch of calendar events."""
    # make sure we have ISO strings
    from datetime import datetime

    if isinstance(start_time, datetime):
        start = start_time.isoformat()
    else:
        start = start_time

    if isinstance(end_time, datetime):
        end = end_time.isoformat()
    else:
        end = end_time
    
    print("-"*30)
    print(f"Fetching events from {start} to {end}")
    
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor() as executor:
        results = await loop.run_in_executor(
            executor,
            lambda: list(executor.map(
                lambda a: (a['email'], retrieve_calendar_events(a['email'], start, end)),
                attendees
            ))
        )
    return {email: (events, tz) for email, (events, tz) in results}

# Usage example:
def process_meeting_request(meeting_request):
    """Process meeting request and fetch events for the specific day only."""
    
    # Extract target day from email content
    target_day = extract_target_day_from_email(meeting_request.get('EmailContent', ''))
    
    # Calculate the specific date range
    start_date, end_date = calculate_meeting_date_range(
        meeting_request['Datetime'], 
        target_day
    )
    
    print(f"Request datetime: {meeting_request['Datetime']}")
    print(f"Target day: {target_day}")
    print(f"Fetching events for: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    
    # Fetch events for attendees
    attendees = meeting_request.get('Attendees', [])
    
    # Add the organizer to attendees list if not already present
    organizer_email = meeting_request.get('From')
    if organizer_email and not any(a.get('email') == organizer_email for a in attendees):
        attendees.append({'email': organizer_email})
    
    # Fetch events asynchronously
    events_data = asyncio.run(fetch_all_events(attendees, start_date, end_date))
    
    return events_data

In [9]:
def is_working_hours(slot_start, tz):
    """Check if slot is within working hours (9 AM–5 PM, excluding 12–2 PM)."""
    slot_start = slot_start.astimezone(tz)
    hour = slot_start.hour
    weekday = slot_start.weekday()
    return (9 <= hour < 12 or 14 <= hour < 17) and weekday < 5  # Weekdays only

def find_free_slots(events, start_time, end_time, duration_mins, time_zone, priority, preferences):
    """Find free time slots with preference consideration."""
    tz = pytz.timezone(time_zone)
    start_time = start_time.astimezone(tz)
    end_time = end_time.astimezone(tz)
    duration = timedelta(minutes=duration_mins)
    free_slots = []
    current_time = start_time if priority != "urgent" else datetime.now(tz)

    events = sorted(events, key=lambda x: parser.parse(x['StartTime']))
    while current_time + duration <= end_time:
        slot_end = current_time + duration
        conflict = False
        for event in events:
            event_start = parser.parse(event['StartTime']).astimezone(tz)
            event_end = parser.parse(event['EndTime']).astimezone(tz)
            if not (slot_end <= event_start or current_time >= event_end):
                conflict = True
                current_time = event_end  # Move to the end of the conflicting event
                break
        if not conflict and is_working_hours(current_time, tz):
            slot_score = 1.0
            if preferences:
                if "morning" in preferences.lower() and current_time.hour < 12:
                    slot_score += 0.5
                elif "afternoon" in preferences.lower() and current_time.hour >= 14:
                    slot_score += 0.5
            free_slots.append({
                "StartTime": current_time.isoformat(),
                "EndTime": slot_end.isoformat(),
                "Score": slot_score
            })
        # Increment by 30 minutes, but check next slot immediately after conflict
        current_time += timedelta(minutes=30)
        # If conflict occurred, try the next slot immediately after event_end
        if conflict:
            current_time = max(current_time, event_end)
    return sorted(free_slots, key=lambda x: (isoparse(x['StartTime']), -x['Score']))

async def resolve_conflicts_async(attendees, requested_time, duration_mins, time_zone, priority, ai_agent, preferences, lang="en"):
    """Resolve conflicts with preference-based slot selection."""
    metadata = {"negotiation_log": [], "excluded_attendees": []}
    tz = pytz.timezone(time_zone)
    start_range = requested_time if priority != "urgent" else datetime.now(tz)
    end_range = requested_time + timedelta(days=7)
    all_free_slots = []
    available_attendees = []

    # Async fetch events
    events_dict = await fetch_all_events(attendees, start_range.isoformat(), end_range.isoformat())
    for attendee in attendees:
        events, attendee_tz = events_dict.get(attendee['email'], ([], time_zone))
        free_slots = find_free_slots(events, start_range, end_range, duration_mins, attendee_tz, priority, preferences)
        if free_slots:
            all_free_slots.append(free_slots)
            available_attendees.append(attendee)
            metadata["negotiation_log"].append(f"Found {len(free_slots)} free slots for {attendee['email']} in {attendee_tz}")
        else:
            metadata["negotiation_log"].append(f"No free slots for {attendee['email']} in {attendee_tz}")
            metadata["excluded_attendees"].append(attendee['email'])

    # Find common slots
    common_slots = []
    if all_free_slots:
        common_slots = all_free_slots[0]
        for slots in all_free_slots[1:]:
            common_slots = [
                slot for slot in common_slots
                if any(parser.parse(slot['StartTime']) == parser.parse(s['StartTime']) for s in slots)
            ]
        metadata["negotiation_log"].append(f"Found {len(common_slots)} common slots across {len(available_attendees)} attendees.")

    if not common_slots:
        reschedule_msg = ai_agent.generate_reschedule_message(
            f"No common slots found. Excluded: {', '.join(metadata['excluded_attendees'])}", lang=lang
        )
        metadata["negotiation_log"].append(f"Reschedule message: {reschedule_msg}")
        metadata["reschedule"] = reschedule_msg
        return None, metadata, available_attendees

    # Select highest-scoring slot on the requested day first
    requested_day = requested_time.date()
    common_slots.sort(key=lambda s: (isoparse(s['StartTime']).date() != requested_day, isoparse(s['StartTime']), -s['Score']))
    selected_slot = common_slots[0]
    metadata["negotiation_log"].append(
        f"Selected slot: {selected_slot['StartTime']}"
    )
    metadata["reschedule"] = ai_agent.generate_reschedule_message(
        f"Scheduled at {selected_slot['StartTime']} with {len(available_attendees)} attendees.",
        lang=lang
    )
    return selected_slot, metadata, available_attendees

In [10]:
async def send_discord_notification(message):
    """Send notification via Discord if enabled."""
    if discord_bot and DISCORD_CHANNEL:
        try:
            channel = discord_bot.get_channel(DISCORD_CHANNEL)
            await channel.send(message)
        except Exception as e:
            logging.error(f"Discord notification error: {e}")
    else:
        logging.debug("Discord notifications disabled")

async def send_slack_notification(message):
    """Send notification via Slack if enabled."""
    if slack_client:
        try:
            await asyncio.to_thread(slack_client.chat_postMessage, channel="general", text=message)
        except Exception as e:
            logging.error(f"Slack notification error: {e}")
    else:
        logging.debug("Slack notifications disabled")

async def send_email_notification(to_emails, subject, body, ics_path=None):
    """Send notification via email with optional .ics attachment if enabled."""
    if EMAIL_SERVER and EMAIL_USER and EMAIL_PASS:
        try:
            msg = MIMEMultipart()
            msg['Subject'] = subject
            msg['From'] = 'scheduler_bot@example.com'
            msg['To'] = ', '.join(to_emails)
            msg.attach(MIMEText(body, 'plain'))
            
            if ics_path and os.path.exists(ics_path):
                with open(ics_path, 'rb') as f:
                    part = MIMEBase('application', 'octet-stream')
                    part.set_payload(f.read())
                encoders.encode_base64(part)
                part.add_header(
                    'Content-Disposition',
                    f'attachment; filename={os.path.basename(ics_path)}'
                )
                msg.attach(part)
            
            await asyncio.to_thread(EMAIL_SERVER.login, EMAIL_USER, EMAIL_PASS)
            await asyncio.to_thread(EMAIL_SERVER.sendmail, 'scheduler_bot@example.com', to_emails, msg.as_string())
        except Exception as e:
            logging.error(f"Email notification error: {e}")
    else:
        logging.debug("Email notifications disabled")

async def send_notifications(attendees, subject, message, ics_path=None):
    """Send notifications across all platforms asynchronously."""
    tasks = [
        send_discord_notification(message),
        send_slack_notification(message),
        send_email_notification([a['email'] for a in attendees], subject, message, ics_path)
    ]
    await asyncio.gather(*tasks, return_exceptions=True)

In [11]:
def generate_ics(meeting_data, attendees, event_start, event_end):
    """Generate .ics file for the meeting."""
    try:
        cal = Calendar()
        event = IcsEvent()
        event.add('summary', meeting_data['Subject'])
        # Ensure event_start and event_end are datetime objects
        event_start = parser.parse(event_start) if isinstance(event_start, str) else event_start
        event_end = parser.parse(event_end) if isinstance(event_end, str) else event_end
        event.add('dtstart', event_start)
        event.add('dtend', event_end)
        event.add('dtstamp', datetime.now())
        for attendee in attendees:
            event.add('attendee', f"mailto:{attendee['email']}")
        cal.add_component(event)
        ics_path = f"meeting_{meeting_data['Request_id']}.ics"
        with open(ics_path, 'wb') as f:
            f.write(cal.to_ical())
        logging.info(f"Generated .ics file: {ics_path}")
        return ics_path
    except Exception as e:
        logging.error(f"Error generating .ics file: {e}")
        return None

In [12]:
def detect_conflicts(proposed_start, proposed_end, all_busy_blocks):
    conflicts = []
    logging.debug(f"Checking conflicts for proposed slot: {proposed_start} to {proposed_end}")
    for busy_start_str, busy_end_str, attendee_email in all_busy_blocks:
        try:
            busy_start = isoparse(busy_start_str)
            busy_end = isoparse(busy_end_str)
            if not (proposed_end <= busy_start or proposed_start >= busy_end):
                logging.debug(f"Conflict found for {attendee_email}: {busy_start_str} to {busy_end_str}")
                conflicts.append({
                    "email": attendee_email,
                    "conflict": {
                        "start": busy_start_str,
                        "end": busy_end_str
                    }
                })
            else:
                logging.debug(f"No conflict for {attendee_email}: {busy_start_str} to {busy_end_str}")
        except Exception as e:
            logging.warning(f"Error parsing busy time for {attendee_email}: {e}")
    
    # Remove duplicates
    seen = set()
    unique_conflicts = []
    for conflict in conflicts:
        conflict_key = (conflict['email'], conflict['conflict']['start'], conflict['conflict']['end'])
        if conflict_key not in seen:
            seen.add(conflict_key)
            unique_conflicts.append(conflict)
    
    logging.debug(f"Total unique conflicts detected: {len(unique_conflicts)}")
    return unique_conflicts

async def your_meeting_assistant_enhanced(data):
    """Enhanced meeting assistant with improved conflict detection and resolution."""
    ai_agent = AI_AGENT(vllm_client=client, groq_client=groq_client, model_path=MODEL_PATH)
    start_time = datetime.now()
    try:
        # Validate input with defaults
        try:
            meeting_request = MeetingRequest(**data)
        except Exception as e:
            logging.warning(f"Invalid input, using defaults: {e}")
            data = MeetingRequest(From=data.get('From', 'shaunkmmendes@gmail.com')).dict() | data
            meeting_request = MeetingRequest(**data)

        # Decode EmailContent
        try:
            meeting_request.EmailContent = meeting_request.EmailContent.encode().decode('utf-8')
        except UnicodeDecodeError:
            logging.warning("Failed to decode EmailContent as UTF-8, falling back to unicode_escape")
            meeting_request.EmailContent = meeting_request.EmailContent.encode().decode('unicode_escape')

        # Parse email content
        email_content = meeting_request.EmailContent
        parsed_slot = AI_AGENT.parse_email(email_content, meeting_request.Datetime)
        
        # Extract proposed time slot
        proposed_start = parsed_slot["start"]
        proposed_end = parsed_slot["end"]
        duration_mins = parsed_slot["duration_mins"]
        preferences = parsed_slot["preferences"]
        
        # Extract attendees
        attendees = [{'email': a.email} for a in meeting_request.Attendees]
        if meeting_request.From not in [a['email'] for a in attendees]:
            attendees.append({'email': meeting_request.From})

        if not attendees:
            return {
                "processed": False,
                "output": data | {"MetaData": {"error": "No participants found", "latency": (datetime.now() - start_time).total_seconds()}}
            }

        # Determine time zone
        time_zone = LOCATION_TO_TIMEZONE.get(meeting_request.Location, 'Asia/Kolkata')
        tz = pytz.timezone(time_zone)
        
        # Ensure proposed times are timezone-aware
        if proposed_start.tzinfo is None:
            proposed_start = tz.localize(proposed_start)
        if proposed_end.tzinfo is None:
            proposed_end = tz.localize(proposed_end)

        # 1) Gather *all* busy blocks for *all* attendees
        target_day = extract_target_day_from_email(meeting_request.EmailContent)
        time_zone = LOCATION_TO_TIMEZONE.get(meeting_request.Location, "Asia/Kolkata")
        start_range, end_range = calculate_meeting_date_range(meeting_request.Datetime,target_day, tz_str=time_zone)
        events_dict = await fetch_all_events(
            attendees,
            start_range.isoformat(),
            end_range.isoformat()
        )
        
        all_busy = []
        for attendee in attendees:
            events, _ = events_dict.get(attendee['email'], ([], time_zone))
            attendee['events'] = events
            all_busy.extend([
                (e["StartTime"], e["EndTime"], attendee['email'])
                for e in events
            ])
        logging.debug(f"All busy blocks: {all_busy}")
        

        # 2) Check for conflicts
        slot_start, slot_end = proposed_start, proposed_end
        negotiation_log = detect_conflicts(slot_start, slot_end, all_busy)
        
        # 3) If there *are* conflicts, call resolve_conflicts to find a new slot
        if negotiation_log:
            logging.info(f"Conflicts detected: {len(negotiation_log)}")
            # call the async, preference‑aware resolver
            selected_slot, conflict_metadata, available = await resolve_conflicts_async(
                attendees,
                proposed_start,
                duration_mins,
                time_zone,
                priority="normal",
                ai_agent=ai_agent,
                preferences=preferences
            )
            if selected_slot:
                new_start = isoparse(selected_slot["StartTime"])
                new_end = isoparse(selected_slot["EndTime"])
                negotiation_log.extend(conflict_metadata["negotiation_log"])
            if 'new_start' in locals() and new_start and new_end:
                # Record rescheduling
                negotiation_log.append({
                    "action": "rescheduled",
                    "original_time": slot_start.isoformat(),
                    "new_time": new_start.isoformat(),
                    "reason": "conflict resolution"
                })
                slot_start, slot_end = new_start, new_end
            else:
                return {
                    "processed": False,
                    "output": data | {"MetaData": {
                        "error": "No slots free within 7 days",
                        "negotiation_log": negotiation_log,
                        "latency": (datetime.now() - start_time).total_seconds()
                    }}
                }
        else:
            logging.info(f"No conflicts, using original time: {slot_start}")

        # Generate and send .ics file
        ics_path = generate_ics(data, attendees, slot_start, slot_end)
        if ics_path:
            notification_message = f"Meeting scheduled for {slot_start.strftime('%Y-%m-%d %H:%M')} ({time_zone})"
            tasks = []
            if discord_bot and DISCORD_CHANNEL:
                tasks.append(send_discord_notification(notification_message))
            if slack_client:
                tasks.append(send_slack_notification(notification_message))
            if EMAIL_SERVER and EMAIL_USER and EMAIL_PASS:
                tasks.append(send_email_notification(
                    [a['email'] for a in attendees], 
                    meeting_request.Subject, 
                    notification_message, 
                    ics_path
                ))
            
            if tasks:
                await asyncio.gather(*tasks, return_exceptions=True)
            else:
                logging.debug("No notifications sent: no platforms enabled")

        conflicts_detected = len([entry for entry in negotiation_log if isinstance(entry, dict) and 'email' in entry])

        # Create output dictionary
        latency = (datetime.now() - start_time).total_seconds()
        output = {
            "Request_id": meeting_request.Request_id,
            "Datetime": meeting_request.Datetime,
            "Location": meeting_request.Location,
            "From": meeting_request.From,
            "Attendees": attendees,
            "Subject": meeting_request.Subject,
            "EmailContent": meeting_request.EmailContent,
            "EventStart": slot_start.isoformat(),
            "EventEnd": slot_end.isoformat(),
            "Duration_mins": str(duration_mins),
            "MetaData": {
                "negotiation_log": negotiation_log,
                "latency": latency,
                "preferences": preferences,
                "conflicts_detected": conflicts_detected,
                "timezone": time_zone
            }
        }

        response = {"processed": True, "output": output}
        logging.info(f"Meeting scheduled successfully. Processing time: {latency:.2f} seconds")
        return response
        
    except Exception as e:
        latency = (datetime.now() - start_time).total_seconds()
        logging.error(f"Error in enhanced meeting assistant: {e}")
        return {
            "processed": False,
            "output": data | {"MetaData": {"error": str(e), "latency": latency}}
        }

In [13]:
@app.route('/receive', methods=['POST'])
def receive():
    """Handle incoming JSON requests."""
    try:
        data = request.get_json()
        if not data:
            raise ValueError("Invalid JSON")
        logging.info(f"Received: {json.dumps(data, indent=2)}")
        request_queue.put(data)
        new_data = asyncio.run(your_meeting_assistant_enhanced(data))
        received_data.append(data)
        logging.info(f"Sending: {json.dumps(new_data, indent=2)}")
        return jsonify(new_data)
    except ValueError as e:
        logging.error(f"Invalid JSON: {e}")
        return jsonify({"processed": False, "output": {"MetaData": {"error": "Invalid JSON"}}})
    except Exception as e:
        logging.error(f"Flask error: {e}")
        return jsonify({"processed": False, "output": {"MetaData": {"error": str(e)}}})

def run_flask():
    app.run(host='0.0.0.0', port=5000)

# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


2025-07-13 09:05:22,459 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.176.241:5000
2025-07-13 09:05:22,459 - INFO - Press CTRL+C to quit
2025-07-13 09:05:22,527 - INFO - Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "12-07-2025T12:34:55",
  "Location": "New York",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}
2025-07-13 09:05:22,554 - WARNING - Parsed time 2025-07-17 00:00:00+05:30 is outside working hours; defaulting to 9 AM next day
2025-07-13 09:05:22,558 - INFO - file_cache is only supported

------------------------------
Fetching events from 2025-07-17T00:00:00-04:00 to 2025-07-17T23:59:59.999999-04:00


2025-07-13 09:05:22,974 - INFO - No conflicts, using original time: 2025-07-18 09:00:00+05:30
2025-07-13 09:05:22,975 - INFO - Generated .ics file: meeting_6118b54f-907b-4451-8d48-dd13d76033a5.ics
2025-07-13 09:05:22,975 - INFO - Meeting scheduled successfully. Processing time: 0.45 seconds
2025-07-13 09:05:22,976 - INFO - Sending: {
  "processed": true,
  "output": {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "12-07-2025T12:34:55",
    "Location": "New York",
    "From": "userone.amd@gmail.com",
    "Attendees": [
      {
        "email": "usertwo.amd@gmail.com",
        "events": [
          {
            "StartTime": "2025-07-17T18:00:00+05:30",
            "EndTime": "2025-07-18T09:00:00+05:30",
            "NumAttendees": 1,
            "Attendees": [
              "SELF"
            ],
            "Summary": "Off Hours"
          }
        ]
      },
      {
        "email": "userthree.amd@gmail.com",
        "events": [
          {
            "Sta

In [14]:
import requests

meeting_request = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "12-07-2025T12:34:55",
    "Location": "New York",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {"email": "usertwo.amd@gmail.com"},
        {"email": "userthree.amd@gmail.com"}
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

try:
    print("Sending request...")
    response = requests.post(
        'http://localhost:5000/receive',
        json=meeting_request,
        headers={'Content-Type': 'application/json'}
    )
    print("Status code:", response.status_code)
    print("Response text:", response.text)  # Print raw response first
    print("JSON response:", response.json())  # Then try parsing as JSON
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
except json.JSONDecodeError as e:
    print("Failed to decode JSON:", e)
    print("Raw response:", response.text)

Sending request...
Status code: 200
Response text: {"output":{"Attendees":[{"email":"usertwo.amd@gmail.com","events":[{"Attendees":["SELF"],"EndTime":"2025-07-18T09:00:00+05:30","NumAttendees":1,"StartTime":"2025-07-17T18:00:00+05:30","Summary":"Off Hours"}]},{"email":"userthree.amd@gmail.com","events":[{"Attendees":["SELF"],"EndTime":"2025-07-18T07:30:00+05:30","NumAttendees":1,"StartTime":"2025-07-17T16:00:00+05:30","Summary":"Off Hours"}]},{"email":"userone.amd@gmail.com","events":[{"Attendees":["SELF"],"EndTime":"2025-07-18T09:00:00+05:30","NumAttendees":1,"StartTime":"2025-07-17T18:00:00+05:30","Summary":"Off Hours"}]}],"Datetime":"12-07-2025T12:34:55","Duration_mins":"30","EmailContent":"Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.","EventEnd":"2025-07-18T09:30:00+05:30","EventStart":"2025-07-18T09:00:00+05:30","From":"userone.amd@gmail.com","Location":"New York","MetaData":{"conflicts_detected":0,"latency":0.447708,"negotiation_log"

In [15]:
# @discord_bot.command()
# async def get_token(ctx, email: str):
#     """Fetch Google Calendar auth token via Discord."""
#     token_path = f"../Keys/{email.split('@')[0]}.token"
#     if os.path.exists(token_path):
#         await ctx.send(f"Token for {email} found.")
#     else:
#         await ctx.send(f"No token for {email}.")
#         logging.error(f"Token not found for {email}")

# @discord_bot.event
# async def on_ready():
#     logging.info(f"Discord bot logged in as {discord_bot.user}")

# def run_discord_bot():
#     discord_bot.run(DISCORD_TOKEN)

# Thread(target=run_discord_bot, daemon=True).start()

In [16]:
# def mock_calendar_events(email, start_time, end_time):
#     """Mock calendar events for testing."""
#     start = parser.parse(start_time)
#     end = parser.parse(end_time)
#     if email == "shaunkmmendes@gmail.com":
#         return [
#             {"StartTime": "2025-07-10T09:00:00+05:30", "EndTime": "2025-07-10T10:00:00+05:30", "NumAttendees": 1, "Attendees": ["SELF"], "Summary": "Personal"},
#             {"StartTime": "2025-07-08T09:00:00+05:30", "EndTime": "2025-07-08T17:00:00+05:30", "NumAttendees": 3, "Attendees": ["shaunkmmendes@gmail.com", "kingshaun2014@gmail.com"], "Summary": "AMD AI Workshop"}
#         ], "Asia/Kolkata"
#     elif email == "kingshaun2014@gmail.com":
#         return [
#             {"StartTime": "2025-07-10T10:00:00+05:30", "EndTime": "2025-07-10T11:00:00+05:30", "NumAttendees": 2, "Attendees": ["kingshaun2014@gmail.com", "dabresharian@gmail.com"], "Summary": "Team Meeting"},
#             {"StartTime": "2025-07-08T09:00:00+05:30", "EndTime": "2025-07-08T17:00:00+05:30", "NumAttendees": 3, "Attendees": ["shaunkmmendes@gmail.com", "kingshaun2014@gmail.com"], "Summary": "AMD AI Workshop"},
#             {"StartTime": "2025-07-09T10:00:00+05:30", "EndTime": "2025-07-09T11:00:00+05:30", "NumAttendees": 2, "Attendees": ["kingshaun2014@gmail.com", "crce.9962.ce@gmail.com"], "Summary": "Customer Meeting"}
#         ], "Asia/Kolkata"
#     elif email == "crce.9962.ce@gmail.com":
#         return [
#             {"StartTime": "2025-07-07T09:00:00+05:30", "EndTime": "2025-07-07T10:00:00+05:30", "NumAttendees": 1, "Attendees": ["SELF"], "Summary": "Daily Standup"},
#             {"StartTime": "2025-07-08T11:00:00+05:30", "EndTime": "2025-07-08T11:30:00+05:30", "NumAttendees": 2, "Attendees": ["crce.9962.ce@gmail.com", "dabresharian@gmail.com"], "Summary": "1:1 Meeting"},
#             {"StartTime": "2025-07-14T09:00:00+05:30", "EndTime": "2025-07-14T10:00:00+05:30", "NumAttendees": 1, "Attendees": ["SELF"], "Summary": "Weekly Review"}
#         ], "Asia/Kolkata"
#     return [], "Asia/Kolkata"


# async def test_meeting_assistant():
#     """Test the assistant with diverse scenarios."""
#     test_cases = [
#     # Test Case 1: Both available
#     {
#         "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
#         "Datetime": "02-07-2025T12:34:55",
#         "Location": "IIT Mumbai",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [{"email": "kingshaun2014@gmail.com"}, {"email": "crce.9962.ce@gmail.com"}],
#         "Subject": "Goals Discussion",
#         "EmailContent": "Hi Team. Let's meet next Thursday and discuss about our Goals, prefer mornings."
#     },
#     # Test Case 2: One busy
#     {
#         "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
#         "Datetime": "02-07-2025T12:34:55",
#         "Location": "IIT Mumbai",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [{"email": "kingshaun2014@gmail.com"}, {"email": "crce.9962.ce@gmail.com"}],
#         "Subject": "Client Feedback",
#         "EmailContent": "Hi Team. We’ve just received quick feedback from the client indicating that the instructions we provided aren’t working on their end. Let’s prioritize resolving this promptly. Let’s meet Monday at 9:00 AM to discuss and resolve this issue."
#     },
#     # Test Case 3: Both busy
#     {
#         "Request_id": "6118b54f-907b-4451-8d48-dd13d76033c5",
#         "Datetime": "02-07-2025T12:34:55",
#         "Location": "IIT Mumbai",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [{"email": "kingshaun2014@gmail.com"}, {"email": "crce.9962.ce@gmail.com"}],
#         "Subject": "Project Update",
#         "EmailContent": "Hi Team. Let's meet on Tuesday at 11:00 A.M and discuss about our on-going Projects."
#     },
#     # Test Case 4: One busy
#     {
#         "Request_id": "d4b2a7f0-1234-4c8d-9eab-abcdef123456",
#         "Datetime": "02-07-2025T12:34:55",
#         "Location": "IIT Mumbai",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [{"email": "kingshaun2014@gmail.com"}, {"email": "crce.9962.ce@gmail.com"}],
#         "Subject": "Final Feedback Review",
#         "EmailContent": "Hi Team. We’ve received the final feedback from the client. Let’s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M."
#     },
#     # Test Case 5: Missing attendees
#     {
#         "Request_id": str(uuid.uuid4()),
#         "Datetime": "12-07-2025T22:50:00",
#         "Location": "Online",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [],
#         "Subject": "Empty Meeting",
#         "EmailContent": "Let's meet tomorrow."
#     },
#     # Test Case 6: Recurring event conflict
#     {
#         "Request_id": str(uuid.uuid4()),
#         "Datetime": "12-07-2025T22:50:00",
#         "Location": "Online",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [{"email": "kingshaun2014@gmail.com"},{"email": "crce.9962.ce@gmail.com"}],
#         "Subject": "Standup Conflict",
#         "EmailContent": "Meet Monday at 9 AM for 30 minutes."
#     },
#         # Test Case 7: "Next Thursday" on a Thursday
#     {
#         "Request_id": str(uuid.uuid4()),
#         "Datetime": "03-07-2025T12:34:55",  # July 3, 2025, a Thursday
#         "Location": "IIT Mumbai",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [{"email": "kingshaun2014@gmail.com"}, {"email": "crce.9962.ce@gmail.com"}],
#         "Subject": "Next Week Test",
#         "EmailContent": "Hi Team. Let's meet next Thursday."
#     },
#         # Test Case 8: Urgent Meeting
#     {
#         "Request_id": str(uuid.uuid4()),
#         "Datetime": "02-07-2025T12:34:55",
#         "Location": "Online",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [{"email": "kingshaun2014@gmail.com"}, {"email": "crce.9962.ce@gmail.com"}],
#         "Subject": "Urgent Issue",
#         "EmailContent": "Meet asap"
#     },
#         # Test Case 9: Non-working hours
#     {
#         "Request_id": str(uuid.uuid4()),
#         "Datetime": "02-07-2025T12:34:55",
#         "Location": "IIT Mumbai",
#         "From": "shaunkmmendes@gmail.com",
#         "Attendees": [{"email": "kingshaun2014@gmail.com"}],
#         "Subject": "Evening Meeting",
#         "EmailContent": "Let's meet tomorrow at 8 PM."
#     },
# ]

#     # Mock calendar events for testing
#     global retrieve_calendar_events
#     retrieve_calendar_events = mock_calendar_events

#     for i, test_case in enumerate(test_cases, 1):
#         print(f"\nTest Case {i}:")
#         print(f"Input: {json.dumps(test_case, indent=2)}")
#         start_time = datetime.now()
#         result = await your_meeting_assistant_enhanced(test_case)
#         print(f"Output: {json.dumps(result, indent=2)}")
#         print(f"Latency: {(datetime.now() - start_time).total_seconds()} seconds")

# # Run tests
# asyncio.run(test_meeting_assistant())